In [5]:
import numpy as np
import subprocess
import pprint
import pandas as pd

from anytree import Node, RenderTree
from anytree.exporter import DotExporter

In [20]:
X = np.array(
    [['youth', 'high', 'no', 'fair'],
    ['youth', 'high', 'no', 'excellent'],
    ['middle_aged', 'high', 'no', 'fair'],
    ['senior', 'medium', 'no', 'fair'],
    ['senior', 'low', 'yes', 'fair'],
    ['senior', 'low', 'yes', 'excellent'],
    ['middle_aged', 'low', 'yes', 'excellent'],
    ['youth', 'medium', 'no', 'fair'],
    ['youth', 'low', 'yes', 'fair'],
    ['senior', 'medium', 'yes', 'fair'],
    ['youth', 'medium', 'yes', 'excellent'],
    ['middle_aged', 'medium', 'no', 'excellent'],
    ['middle_aged', 'high', 'yes', 'fair'],
    ['senior', 'medium', 'no', 'excellent']]
)

Y = np.array(['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no'])

X_learn = X[:-4]
Y_learn = Y[:-4]


X_test = X[-4:]
Y_test = Y[-4:]

column_labels = ['age', 'income', 'student', 'credit_rating']
df = pd.DataFrame(X_learn, columns=column_labels)

In [32]:
def create_node(
    prop_name = None,
    left_value = None,
    right_value = None,
    left_node = None,
    right_node = None,
    gini_score = None,
    information_gain = None,
    belongs_to_class = None):
  return {
      'prop_name': prop_name,
      'left_value': left_value,
      'right_value': right_value,
      'left_node': left_node,
      'right_node': right_node,
      'gini_score': gini_score,
      'information_gain': information_gain,
      'belongs_to_class': belongs_to_class
  }

def calculate_gini_index(Y):
  classes = np.unique(Y)
  result = 1
  for c in classes:
    p = Y[Y == c].size / Y.size
    result -= p**2
  return result

def calculate_entropy(Y):
    entropy = 0
    y_classes = np.unique(Y)
    for y_class in y_classes:
        amount = Y[Y == y_class].size
        entropy -= np.log2(amount/Y.size)*amount/Y.size
    return entropy
    

def calculate_information_gain(X, Y):
    info_gain = calculate_entropy(Y)
    for x_class in np.unique(X):
        filtered_Y = Y[X == x_class]
        class_entropy = calculate_entropy(filtered_Y)
        info_gain -= class_entropy * filtered_Y.size/Y.size
    return info_gain
        

def split_data(X, Y, node):
    chosen_X = None
    min_gini = 1
    information_gain = None
    left_X_class = None
    right_X_class = None
    for column in X.columns:
        X_i = X[column]
        classes = np.unique(X_i)

        local_min = 1
        local_left_class = None
        local_right_class = None

        for c in classes:
            left_Y = Y[X_i != c]
            right_Y = Y[X_i == c]
            left_gini = calculate_gini_index(left_Y)
            right_gini = calculate_gini_index(right_Y)
            local_gini = (left_Y.size / Y.size) * left_gini + (right_Y.size / Y.size) * right_gini
            if local_gini < local_min:
                local_min = local_gini
                local_left_class = np.unique(X[X[column] != c][column])
                local_right_class = [c]

        if local_min < min_gini:
          min_gini = local_min
          chosen_X = column
          information_gain = calculate_information_gain(X_i, Y)
          left_X_class = local_left_class
          right_X_class = local_right_class

    # these are for when Y is already clear for some branch
    left_class = None
    right_class = None


    unique_Y_left = np.unique(Y[np.isin(X[chosen_X], left_X_class)])
    unique_Y_right = np.unique(Y[np.isin(X[chosen_X], right_X_class)])

    left_class = unique_Y_left[0] if unique_Y_left.size == 1 else None
    right_class = unique_Y_right[0] if unique_Y_right.size == 1 else None


    return min_gini, chosen_X, left_X_class, right_X_class, left_class, right_class, information_gain

def create_tree(X, Y, root = create_node()):
    if root['belongs_to_class'] != None or X.size == 0:
        return root

    min_gini, chosen_X, left_X_class, right_X_class, left_class, right_class, information_gain = split_data(X, Y, root)

    if left_class != None:
        mask = np.isin(X[chosen_X], left_X_class)
        X = X[~mask]
        Y = Y[~mask]


    if right_class != None:
        mask = np.isin(X[chosen_X], right_X_class)
        X = X[~mask]
        Y = Y[~mask]

    right_mask = np.isin(X[chosen_X], right_X_class)
    left_X = X[~right_mask]
    right_X = X[right_mask]
    left_Y = Y[~right_mask]
    right_Y = Y[right_mask]

    left_node = create_node(
        gini_score = "{:.2f}".format(min_gini),
        information_gain = "{:.2f}".format(information_gain),
        belongs_to_class = left_class)
    right_node = create_node(
        gini_score = "{:.2f}".format(min_gini),
        information_gain = "{:.2f}".format(information_gain),
        belongs_to_class = right_class
    )

    root['left_node'] = create_tree(left_X, left_Y, left_node)
    root['right_node'] = create_tree(right_X, right_Y, right_node)
    # root['gini_score'] = "{:.2f}".format(min_gini)
    # root['information_gain'] = "{:.2f}".format(information_gain)
    root['prop_name'] = chosen_X
    root['left_X_class'] = left_X_class
    root['right_X_class'] = right_X_class


    return root

In [34]:
root = create_tree(df, Y_learn)

In [35]:
i = 0
def create_tree_nodes(data, parent_node=None, edge_label=None):
    if data is None:
        return None
    global i
    i += 1
    if data.get('belongs_to_class') is not None:
        prop_name = data['belongs_to_class']
    else:
        prop_name = data.get('prop_name', 'not defined')

    node = Node(f'{str(prop_name)} {str(i)} GI:{data.get('gini_score', 'N/A')} IG:{data.get('information_gain', 'N/A')}', parent=parent_node,
                  edgeattr={'label': str(edge_label)} if edge_label is not None else None)

    create_tree_nodes(data.get('left_node'), parent_node=node, edge_label=data.get('left_X_class'))
    create_tree_nodes(data.get('right_node'), parent_node=node, edge_label=data.get('right_X_class'))

    return node

root_node = create_tree_nodes(root)

print(RenderTree(root_node))

dot_path = "tree_visualization.dot"
DotExporter(
    root_node,
    nodenamefunc=lambda node: node.name,
    edgeattrfunc=lambda parent, child: 'label="%s"' % (child.edgeattr.get('label') if child.edgeattr is not None else '')
).to_dotfile(dot_path)

png_path = "tree_visualization.png"
subprocess.check_call(["dot", dot_path, "-T", "png", "-o", png_path])

Node('/age 1 GI:None IG:None', edgeattr=None)
├── Node('/age 1 GI:None IG:None/credit_rating 2 GI:0.32 IG:0.32', edgeattr={'label': "['middle_aged' 'senior']"})
│   ├── Node('/age 1 GI:None IG:None/credit_rating 2 GI:0.32 IG:0.32/yes 3 GI:0.17 IG:0.32', edgeattr={'label': "['fair']"})
│   └── Node('/age 1 GI:None IG:None/credit_rating 2 GI:0.32 IG:0.32/age 4 GI:0.17 IG:0.32', edgeattr={'label': "['excellent']"})
│       ├── Node('/age 1 GI:None IG:None/credit_rating 2 GI:0.32 IG:0.32/age 4 GI:0.17 IG:0.32/no 5 GI:0.00 IG:1.00', edgeattr={'label': "['senior']"})
│       └── Node('/age 1 GI:None IG:None/credit_rating 2 GI:0.32 IG:0.32/age 4 GI:0.17 IG:0.32/yes 6 GI:0.00 IG:1.00', edgeattr={'label': "['middle_aged']"})
└── Node('/age 1 GI:None IG:None/income 7 GI:0.32 IG:0.32', edgeattr={'label': "['youth']"})
    ├── Node('/age 1 GI:None IG:None/income 7 GI:0.32 IG:0.32/no 8 GI:0.00 IG:0.81', edgeattr={'label': "['high' 'medium']"})
    └── Node('/age 1 GI:None IG:None/income 7 GI:0.32 I

0

In [36]:
def predict(root, X_frame):
  if root['belongs_to_class'] != None:
      return root['belongs_to_class']

  if X_frame[root['prop_name']] in root['left_X_class']:
      return predict(root['left_node'], X_frame)
  else:
      return predict(root['right_node'], X_frame)

In [37]:
Y_predict = [predict(root, x[1]) for x in pd.DataFrame(X_test, columns=column_labels).iterrows()]

print(Y_predict)
print(Y_test)
print("accuracy", (Y_predict == Y_test).sum() / Y_test.size)

[np.str_('no'), np.str_('yes'), np.str_('yes'), np.str_('no')]
['yes' 'yes' 'yes' 'no']
accuracy 0.75


In [40]:
# this is just for testing that info gain is calculated correctly

data = [
    ['Sunny', 'Hot', 'High', 'Weak'],
    ['Sunny', 'Warm', 'Normal', 'Weak'],
    ['Overcast', 'Cold', 'Normal', 'Strong'],
    ['Sunny', 'Warm', 'Normal', 'Strong'],
    ['Rainy', 'Warm', 'High', 'Weak'],
    ['Overcast', 'Hot', 'Normal', 'Strong'],
    ['Rainy', 'Warm', 'High', 'Strong'],
    ['Sunny', 'Cold', 'High', 'Weak'],
    ['Rainy', 'Cold', 'High', 'Weak'],
    ['Overcast', 'Hot', 'Normal', 'Strong'],
    ['Rainy', 'Cold', 'Normal', 'Weak'],
    ['Sunny', 'Hot', 'High', 'Weak']
]

data_y = np.array(['No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No'])

columns = ['Outlook', 'Temperature', 'Humidity', 'Wind']

test_df = pd.DataFrame(data, columns=columns)

print(calculate_information_gain(test_df['Temperature'], data_y))

0.37610938183144177
